#**Natural Language Processing-Homework1**

####**Student name** : Amirahmad Shafiee<br/>**Student number** : 99104027<br/>**Chosen task** : classification
In this notebook we aim to do common but rather essential preprocess actions on some text chosen from the philosophical world. After doing so, an NLP task would be operated on the processed text.

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


##Essential packages and data:

The data we tend to use in this notebook will be the texts extracted from philosophers books on ethics, like : 
>**Immanuel Kant's** "Critique of Pure Reason"<br/>
>**Friedrich Neitzsche's**"Human all too human"<br/>
>**Georg Wilhelm Friedrich Hegel's**"Phenomenology of spirit"<br/>

In [ ]:
#ignore the cell if text file is available
!gdown https://rauterberg.employee.id.tue.nl/lecturenotes/DDM110%20CAS/Kant-1781%20Critique%20of%20Pure%20Reason.pdf
!gdown http://www.faculty.umb.edu/gary_zabel/Courses/Marxist_Philosophy/Hegel_and_Feuerbach_files/Hegel-Phenomenology-of-Spirit.pdf
!gdown https://drive.google.com/file/d/1x4cM62wac-nEU8X_rEyYLLsWOX67Kqmr/view?usp=share_link

Downloading...
From: https://rauterberg.employee.id.tue.nl/lecturenotes/DDM110%20CAS/Kant-1781%20Critique%20of%20Pure%20Reason.pdf
To: /content/gdrive/MyDrive/NLP/Kant-1781%20Critique%20of%20Pure%20Reason.pdf
100% 10.2M/10.2M [00:00<00:00, 41.5MB/s]
Downloading...
From: http://www.faculty.umb.edu/gary_zabel/Courses/Marxist_Philosophy/Hegel_and_Feuerbach_files/Hegel-Phenomenology-of-Spirit.pdf
To: /content/gdrive/MyDrive/NLP/Hegel-Phenomenology-of-Spirit.pdf
100% 34.0M/34.0M [00:01<00:00, 21.3MB/s]
/usr/local/lib/python3.9/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1x4cM62wac-nEU8X_rEyYLLsWOX67Kqmr
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1x4cM62wac-nEU8X_rEyYLLsWOX67Kqmr/view?usp=share_link
To: /content/gdrive/MyDrive/NLP/view?usp=share_link
75.2kB [00:00, 25.1MB/s]


Our data comes in pdf form. So we need the primary text extractors compatible with pdf format.

In [ ]:
!pip install -q openpyxl==3.0.10
!pip install -q PyPDF2==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 15.4 MB/s eta 0:00:00


In [ ]:
import random 
import tqdm
import nltk, re
from nltk import word_tokenize, sent_tokenize, WhitespaceTokenizer, FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist

import itertools
import string
import numpy as np
import pandas as pd

from tqdm import tqdm
import pickle

import plotly.express as px
import plotly.graph_objects as go

from PyPDF2 import PdfReader

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

##Section A: Extracting data



In [ ]:
name_list=["heg","kan","nei"]
address_list=["Hegel-Phenomenology-of-Spirit.pdf","Kant-1781%20Critique%20of%20Pure%20Reason.pdf","Human, All Too Human_ A Book for Free Spirits - PART I + PART II - Friedrich Nietzsche - PDF.pdf"]

In [ ]:
mask_list1 = np.concatenate((np.arange(36),np.array([529]),np.arange(530,631)))
with open('pdf_mask_list_heg.npy','wb') as f:
  np.save(f,mask_list1)

mask_list2 = np.concatenate((np.arange(103),np.array([129,156,625]),np.arange(705,785)))
with open('pdf_mask_list_kan.npy','wb') as f:
  np.save(f,mask_list2)

mask_list3 = np.concatenate((np.arange(30),np.array([230,231,232,233,325]),np.arange(421,425)))
with open('pdf_mask_list_nei.npy','wb') as f:
  np.save(f,mask_list3)

In [ ]:
#ignore if text files are available
for i in range(3):
  reader = PdfReader(address_list[i])
  number_of_pages = len(reader.pages)

  with open(f'pdf_mask_list_{name_list[i]}.npy','rb') as f:
    mask_list = np.load(f)
    text_pages_index = np.delete(np.arange(number_of_pages),mask_list)

  text = ""
  for ind in text_pages_index:
    text+=reader.pages[int(ind)].extract_text()

  with open(f'final_text_{name_list[i]}.txt','wb') as f:
    pickle.dump(text,f)  

In [ ]:
texts = []
for x in name_list:
  with open(f'final_text_{x}.txt','rb') as f:
    text = pickle.load(f)
    texts.append(text)

In [ ]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = [sent_detector.tokenize(text) for text in texts]

In [ ]:
data = pd.DataFrame()
labels=np.array([])
labels_auth = []
for i in range(3):
  data = data.append(sentences[i],ignore_index = True)
  labels = np.concatenate((labels, (i*np.ones(len(sentences[i])))))

labels = labels.astype(np.int)
labels_auth = [name_list[i] for i in labels]
data.rename(columns = {0 : "sentences"}, inplace=True)
data["labels"] = labels
data["author"] = labels_auth

<ipython-input-9-605b0474b0a1>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(sentences[i],ignore_index = True)
<ipython-input-9-605b0474b0a1>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = labels.astype(np.int)


In [ ]:
data.sample(5)

,sentences,labels,author
18721,64 \nHard-hearted out of vanity.-Just as justi...,2,nei
9255,The transcendental unity of apperception is th...,1,kan
4097,In it there is no caprice \nand equally no str...,0,heg
4488,The reflection which is presented in the \njud...,0,heg
1017,Yet; as thus opposed to .one another they \nca...,0,heg


In [ ]:
for txt,x,sen in zip(texts,name_list,sentences):
  print(f"{x} text consists of {len(txt)} characters, in other words the volume of the text is about {len(txt)/1e6:.2f} megs\nalso {len(sen)} sentences have been spotted")

heg text consists of 1251449 characters, in other words the volume of the text is about 1.25 megs
also 7317 sentences have been spotted
kan text consists of 1642328 characters, in other words the volume of the text is about 1.64 megs
also 8148 sentences have been spotted
nei text consists of 1102859 characters, in other words the volume of the text is about 1.10 megs
also 5997 sentences have been spotted


In [ ]:
class distributions:

  def minmax(data, col_end = 'sentence_len_by_words', col_beg = 'sentences'):
    data[col_end] = data[col_beg].apply(lambda t: len(word_tokenize(t)))

    min_max_len = data[col_end].min(), data[col_end].max()
    print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

  def word_freq(data, col_end = "sentence_len_by_words"):

      fig = go.Figure()

      fig.add_trace(go.Histogram(
          x=data[col_end]
      ))

      fig.update_layout(
          title_text='Distribution of word counts within data',
          xaxis_title_text='Word Count',
          yaxis_title_text='Frequency',
          bargap=0.2,
          bargroupgap=0.2)

      fig.show()

  def data_gl_than(data, less_than=25.0, greater_than=3.0, col='sentence_len_by_words'):
      data_length = data[col].values

      data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

      data_glt_rate = (data_glt / len(data_length)) * 100

      print(f'sentences with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

  def full_df_run(data, col_begin = 'sentences', minmax = True, freq = True, gl = True):
    if minmax :
      distributions.minmax(data)
    if freq:
      distributions.word_freq(data)
    if gl:
      distributions.data_gl_than(data)

  def word_and_freq(data, col = "tokenized_sents"):
    tokenized_sent = data[col].values
    total_len = 0
    for i in range(len(tokenized_sent)):
      total_len += len(tokenized_sent[i])

    mp_freqdist = FreqDist(itertools.chain(*tokenized_sent))
    top20words=mp_freqdist.most_common(20)
    print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
    for topword in top20words:
        percent=(topword[1]/total_len)*100
        print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)
    
  def author_count(data, col_auth):
    fig = go.Figure()

    groupby_rate = data.groupby(col_auth)[col_auth].count()

    fig.add_trace(go.Bar(
        x=list(sorted(groupby_rate.index)),
        y=groupby_rate.tolist(),
        text=groupby_rate.tolist(),
        textposition='auto'
    ))

    fig.update_layout(
        title_text='Distribution of authors within sentences',
        xaxis_title_text='author',
        yaxis_title_text='Frequency',
        bargap=0.2,
        bargroupgap=0.2)

    fig.show()

In [ ]:
distributions.full_df_run(data)

Min: 1 	Max: 340


sentences with word length of greater than 3.0 and less than 25.0 includes 31.83% of the whole!


which clearly is not a result we were looking forward to have. Next; we will clean our text. Numbers and oddly nonsentences considered as one, will be eliminated.

##Section B: Text preprocessing

###Subsection 1: tokenizing

In [ ]:
tokenized_sents = [word_tokenize(sent) for sent in data["sentences"]]
data["tokenized_sents"] = tokenized_sents
data.sample(5)

,sentences,labels,author,sentence_len_by_words,tokenized_sents
20946,-\nThis is the life of modern men: they are in...,2,nei,27,"[-, This, is, the, life, of, modern, men, :, t..."
20802,In dreaming \nwe use up too much of our artist...,2,nei,23,"[In, dreaming, we, use, up, too, much, of, our..."
1681,And it is only through staking one's \nlife th...,0,heg,80,"[And, it, is, only, through, staking, one, 's,..."
12586,"In the first edition: "" ... ein schlechthin no...",1,kan,23,"[In, the, first, edition, :, ``, ..., ein, sch..."
16620,"-Because we think well of ourselves, but \n• H...",2,nei,91,"[-Because, we, think, well, of, ourselves, ,, ..."


In [ ]:
distributions.word_and_freq(data,"tokenized_sents")

word             Frequency        % of the total  
,                44766            5.867280402947926
the              42530            5.574217833565101
of               31031            4.06709507626049
.                20465            2.682256476931808
and              16988            2.226541560230518
is               16958            2.222609593736115
to               16331            2.1404314940030957
in               16298            2.1361063308592527
a                13119            1.7194489480023643
it               10440            1.3683243400521903
that             10065            1.3191747588721547
as               9209             1.2069826482318602
which            7064             0.9258470438820567
this             6215             0.8145723920904562
not              6163             0.807756983500158
be               5876             0.770141170703704
its              5436             0.7124723287857957
for              5157             0.6759050403878492
;   

**In the next subsection** we will check this again after normalising our data, actions like:
>omitting stopwords<br/>
>punctuations removal

###Subsection 2: normalising

In [ ]:
Stop_words = stopwords.words('english')
Stop_words[0:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [ ]:
def clean_text(text):
    #removing roman numbers
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    re.sub(pattern, '&', text)
    #removing numbers
    text_nonum = re.sub(r'\d+', '', text)
    #getting rid of punctuations
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    #getting rid of white spaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()

    return text_no_doublespace

def normalize_sentence(tokenized_sents, minimum_length=2, stopword_removal=True, lower_case=False, number_removal = True):
  
    normalized_sents = tokenized_sents
    
    if stopword_removal:
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_sents=[[word for word in sentence if (word.lower() not in stopwords)] for sentence in normalized_sents ]

    if lower_case:
        normalized_sents=[[word.lower() for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    elif minimum_length>1:
        normalized_sents= [[word for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]        
        
    return normalized_sents

In [ ]:
clean_token = [word_tokenize(clean_text(txt)) for txt in data["sentences"]]
data["clean_token"] = clean_token
normalized_sents = normalize_sentence(clean_token)
data["normalized_sents"] = normalized_sents

In [ ]:
data.sample(5)

,sentences,labels,author,sentence_len_by_words,tokenized_sents,clean_token,normalized_sents
15677,"--In our primary condition, all \nthat interes...",2,nei,27,"[--, In, our, primary, condition, ,, all, that...","[in, our, primary, condition, all, that, inter...","[primary, condition, interests, organic, being..."
5024,Insight also finds \nit wrong to deny oneself ...,0,heg,86,"[Insight, also, finds, it, wrong, to, deny, on...","[insight, also, finds, it, wrong, to, deny, on...","[insight, also, finds, wrong, deny, oneself, m..."
12502,"a Antithese; following Erdmann, we read These.",1,kan,10,"[a, Antithese, ;, following, Erdmann, ,, we, r...","[a, antithese, following, erdmann, we, read, t...","[antithese, following, erdmann, read]"
11726,Thereby the expression that only souls (as a p...,1,kan,69,"[Thereby, the, expression, that, only, souls, ...","[thereby, the, expression, that, only, souls, ...","[thereby, expression, souls, par­, ticular, sp..."
11275,That there are three angles in a figure enclos...,1,kan,31,"[That, there, are, three, angles, in, a, figur...","[that, there, are, three, angles, in, a, figur...","[three, angles, figure, enclosed, three, strai..."


In [ ]:
distributions.word_and_freq(data,"normalized_sents")

word             Frequency        % of the total  
one              3646             1.177854018465753
reason           1799             0.5811737189303043
pure             1776             0.5737434823903393
would            1731             0.5592060630730167
consciousness    1648             0.5323926007766213
thus             1562             0.5046099771924044
object           1554             0.5020255470915469
time             1505             0.48619591272379553
must             1324             0.4277231816918972
even             1270             0.41027827851110976
first            1231             0.39767918176943007
also             1184             0.3824956549268929
nature           1151             0.37183488076085625
world            1151             0.37183488076085625
something        1105             0.35697440768092625
therefore        1095             0.3537438700548545
merely           1080             0.34889806361574693
concept          1075             0.347282

In [ ]:
data["sentence_len_by_words"] = [len(sen) for sen in data["normalized_sents"]]
distributions.data_gl_than(data)

sentences with word length of greater than 3.0 and less than 25.0 includes 64.76% of the whole!


**last step of the subsection** we will remove senteces of length 0.

In [ ]:
data = data[data["sentence_len_by_words"]>0]
data.reset_index(inplace=True)

###Subsection 3: Frequency analysis

In [ ]:
distributions.full_df_run(data, minmax=False)
distributions.author_count(data, "author")

sentences with word length of greater than 3.0 and less than 25.0 includes 71.64% of the whole!


In [ ]:
for name in name_list:
  df = data[data["author"]==name]
  print(f"\nword frequency within {name}'s text is as follows:\n")
  distributions.word_and_freq(df,"normalized_sents")


word frequency within heg's text is as follows:

word             Frequency        % of the total  
consciousness    1455             1.5899032945418785
spirit           842              0.9200677484565372
pure             789              0.8621537452876576
self             787              0.8599683112058132
one              739              0.8075178932415451
universal        690              0.7539747582363546
notion           601              0.6567229415942742
essence          600              0.6556302245533518
object           572              0.6250341474075288
existence        545              0.595530787302628
content          526              0.5747691635251051
form             516              0.5638419931158827
therefore        512              0.5594711249521936
world            491              0.5365240670928263
selfconsciousness 457              0.4993716877014697
action           456              0.49827897066054744
nature           448              0.48953723433316

###Subsection 5: lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()
lem_sents = []
for nsent in data["normalized_sents"]:
  tokens = []
  for token in nsent:
    tokens.append(lemmatizer.lemmatize(token))
  lem_sents.append(tokens)
data["lemmatized"] = lem_sents

In [ ]:
data.sample(5)

,index,sentences,labels,author,sentence_len_by_words,tokenized_sents,clean_token,normalized_sents,lemmatized
1331,1476,What was supposed to berhe self-identical is t...,0,heg,12,"[What, was, supposed, to, berhe, self-identica...","[what, was, supposed, to, berhe, selfidentical...","[supposed, berhe, selfidentical, thus, already...","[supposed, berhe, selfidentical, thus, already..."
7772,8726,Div.,1,kan,1,"[Div, .]",[div],[div],[div]
5473,6238,"AccDrdingly, if the various ideas within a par...",0,heg,20,"[AccDrdingly, ,, if, the, various, ideas, with...","[accdrdingly, if, the, various, ideas, within,...","[accdrdingly, various, ideas, within, particul...","[accdrdingly, various, idea, within, particula..."
14649,16692,174 \nDiminution.,2,nei,1,"[174, Diminution, .]",[diminution],[diminution],[diminution]
19037,21096,If we knew all that is irrepar­\nable and hope...,2,nei,12,"[If, we, knew, all, that, is, irrepar­, able, ...","[if, we, knew, all, that, is, irrepar­, able, ...","[knew, irrepar­, able, hopeless, whole, much, ...","[knew, irrepar­, able, hopeless, whole, much, ..."


##Section D: author classification

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.5 MB/s eta 0:00:00


In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F
import os

###Balancing data

In [ ]:
indices_to_keep = [i for i in range(len(data)) if ((len(data["normalized_sents"][i])<30) and len(data["normalized_sents"][i])>2)]
train_data = data.iloc[indices_to_keep]
train_data = train_data[["normalized_sents" , "labels","author"]]
train_data.sample(5)

,normalized_sents,labels,author
16113,"[man, neighs, laughter, excels, animal, vulgar...",2,nei
7171,"[however, assert, absolute, reality, space, ti...",1,kan
16636,"[injustice, perpetrated, much, harder, bear, i...",2,nei
13527,"[yet, attained, laborious, struggle, certain, ...",2,nei
4124,"[yet, plain, mind, diderot]",0,heg


In [ ]:
distributions.author_count(train_data, "author")

In [ ]:
data0 = train_data[train_data['labels'] == 0]
data1 = train_data[train_data['labels'] == 1]
data2 = train_data[train_data['labels'] == 2]


cutting_point = 4512

if cutting_point <= len(data0):
    data0 = data0.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data1):
    data1 = data1.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data2):
    data2 = data2.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([data0,data1,data2])
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [ ]:
distributions.author_count(new_data, "author")

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(new_data, test_size=0.2, random_state=1, stratify=new_data['author'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['author'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['normalized_sents'].values.tolist(), train['author'].values.tolist()
x_valid, y_valid = valid['normalized_sents'].values.tolist(), valid['author'].values.tolist()
x_test, y_test = test['normalized_sents'].values.tolist(), test['author'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(8662, 3)
(2166, 3)
(2708, 3)


### Configuration

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 2
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'bert-base-uncased'
OUTPUT_PATH = '/auth_classification_bert/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
# create a key finder based on label 2 id and id to label

label2id = {label: i for i, label in enumerate(name_list)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'heg': 0, 'kan': 1, 'nei': 2}
id2label: {0: 'heg', 1: 'kan', 2: 'nei'}


In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "heg",
    "1": "kan",
    "2": "nei"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "heg": 0,
    "kan": 1,
    "nei": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



### Input Embeddings

In [ ]:
idx = np.random.randint(0, len(train))
sample_sent = train.iloc[idx]['normalized_sents']
sample_auth = train.iloc[idx]['author']

print(f'Sample: \n{" ".join(sample_sent)}\n{sample_auth}')

Sample: 
philosophy thus become tragedy
nei


In [ ]:
tokens = sample_sent
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  sentence: {" ".join(sample_sent)}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  sentence: philosophy thus become tragedy
  Coded_tokens: ['philosophy', 'thus', 'become', 'tragedy']
   Tokens: philosophy thus become tragedy
Token IDs: [4695, 2947, 2468, 10576]


In [ ]:
encoding = tokenizer.encode_plus(
    sample_sent,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[  101,  4695,  2947,  2468, 10576,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [ ]:
class PhilDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, sent, targets=None, label_list=None, max_len=128):
        self.sent = sent
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.sent)

    def __getitem__(self, item):
        sentence = str(self.sent[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'sentence': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = PhilDataset(
        sent=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
label_list = name_list
train_data_loader = create_data_loader(train['normalized_sents'].to_numpy(), train['labels'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['normalized_sents'].to_numpy(), valid['labels'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['normalized_sents'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['sentence'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
["['anyone', 'declared', 'another', 'idiot', 'unpleasant', 'fellow', 'annoyed', 'end', 'demonstrates', 'one']", "['moral', 'self', 'con\\xad', 'sciousness', 'indeed', 'let', 'universality', 'free', 'becomes', 'nature', 'ofits', 'equallyit', 'holds', 'fast', 'toit', 'within', 'superseded', 'moment']", "['superfluous', 'beings', 'even', 'supposing', 'eqst']", "['brain', 'spinal', 'cord', 'together', 'constitute', 'corporeal', 'forselj', 'spirit', 'skull', 'vertebra', 'column', 'form', 'extreme', 'extreme', 'separated', 'viz']", "['could', 'despondent', 'becomes', 'aware', 'goal', 'paths', 'lead']", "['wealth', 'thus', 'shares', 'dejection', 'recipient', 'place', 'rebellion', 'appears', 'arrogance']", "['state', 'fact', 'thing', 'every', 'father', 'raises', 'son', 'fashion', 'regard', 'true', 'says', 'see', 'much', 'good']", "['worker', 'free', 'choose', 'whether', 'works', 'works']", "['however', 'exist\\

In [ ]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids'])


In [ ]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 10)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)['pooler_output']
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [ ]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [ ]:
for name,param in pt_model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

### Training

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

In [ ]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb

In [ ]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/2 [00:00<?, ?it/s]

Training... :   0%|          | 0/542 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/136 [00:00<?, ?it/s]

Training... :   0%|          | 0/542 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/136 [00:00<?, ?it/s]

Epoch: 2/2...Step: 1000...Train Loss: 0.189957...Train Acc: 0.939...Valid Loss: 0.264137...Valid Acc: 0.915...
Validation loss decreased (inf --> 0.264137).  Saving model ...


Evaluation... :   0%|          | 0/136 [00:00<?, ?it/s]

### Prediction

In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_sents = test['normalized_sents'].to_numpy()
preds, probs = predict(pt_model, test_sents, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/85 [00:00<?, ?it/s]

(2708,) (2708, 10)


In [ ]:
preds

array([2, 1, 0, ..., 1, 1, 1])

In [ ]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['nei', 'kan', 'heg', 'heg', 'heg']

###F1 Score and report

In [ ]:
y_test, y_pred = test['labels'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.9217081269086359
--------------classification_report---------------
              precision    recall  f1-score   support

         heg       0.93      0.92      0.92       902
         kan       0.92      0.93      0.93       903
         nei       0.91      0.91      0.91       903

    accuracy                           0.92      2708
   macro avg       0.92      0.92      0.92      2708
weighted avg       0.92      0.92      0.92      2708



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[827  36  39]
 [ 20 843  40]
 [ 42  35 826]]


good results have been obtained, but I don't see them fully dependent on the preprocess part.

##Section E: running the model without preprocessed data



In [ ]:
indices_to_keep = [i for i in range(len(data)) if ((len(data["tokenized_sents"][i])<30) and len(data["tokenized_sents"][i])>2)]
train_data = data.iloc[indices_to_keep]
train_data = train_data[["tokenized_sents", "labels", "author"]]
train_data.sample(5)

,tokenized_sents,labels,author
7222,"[You, must, therefore, give, your, object, a, ...",1,kan
4738,"[What, consciousness, did, not, know, would, h...",0,heg
15847,"[of, the, masses, :, thus, are, they, all, the...",2,nei
4305,"[SPIRIT, of, consciousness, ,, or, the, pure, ...",0,heg
18904,"[223, Plunder, .md, economy, .]",2,nei


In [ ]:
distributions.author_count(train_data, "author")

In [ ]:
data0 = train_data[train_data['labels'] == 0]
data1 = train_data[train_data['labels'] == 1]
data2 = train_data[train_data['labels'] == 2]


cutting_point = 2550

if cutting_point <= len(data0):
    data0 = data0.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data1):
    data1 = data1.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data2):
    data2 = data2.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([data0,data1,data2])
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [ ]:
distributions.author_count(new_data, "author")

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(new_data, test_size=0.2, random_state=1, stratify=new_data['author'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['author'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['tokenized_sents'].values.tolist(), train['author'].values.tolist()
x_valid, y_valid = valid['tokenized_sents'].values.tolist(), valid['author'].values.tolist()
x_test, y_test = test['tokenized_sents'].values.tolist(), test['author'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(4896, 3)
(1224, 3)
(1530, 3)


### Configuration

In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 2
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'bert-base-uncased'
OUTPUT_PATH = '/auth_classification_bert/pytorch_model_nonprocessed.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
idx = np.random.randint(0, len(train))
sample_sent = train.iloc[idx]['tokenized_sents']
sample_auth = train.iloc[idx]['author']

print(f'Sample: \n{" ".join(sample_sent)}\n{sample_auth}')

Sample: 
In that vanity ; all content is turned into something negative which can no longer be grasped as having a positive significance .
heg


In [ ]:
tokens = sample_sent
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  sentence: {" ".join(sample_sent)}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  sentence: In that vanity ; all content is turned into something negative which can no longer be grasped as having a positive significance .
  Coded_tokens: ['In', 'that', 'vanity', ';', 'all', 'content', 'is', 'turned', 'into', 'something', 'negative', 'which', 'can', 'no', 'longer', 'be', 'grasped', 'as', 'having', 'a', 'positive', 'significance', '.']
   Tokens: In that vanity ; all content is turned into something negative which can no longer be grasped as having a positive significance .
Token IDs: [100, 2008, 18736, 1025, 2035, 4180, 2003, 2357, 2046, 2242, 4997, 2029, 2064, 2053, 2936, 2022, 15517, 2004, 2383, 1037, 3893, 7784, 1012]


In [ ]:
encoding = tokenizer.encode_plus(
    sample_sent,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[  101,   100,  2008, 18736,  1025,  2035,  4180,  2003,  2357,  2046,
          2242,  4997,  2029,  2064,  2053,  2936,  2022, 15517,  2004,  2383,
          1037,  3893,  7784,  1012,   102,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [ ]:
label_list = name_list
train_data_loader = create_data_loader(train['tokenized_sents'].to_numpy(), train['labels'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['tokenized_sents'].to_numpy(), valid['labels'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['tokenized_sents'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [ ]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


### Training

In [ ]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/2 [00:00<?, ?it/s]

Training... :   0%|          | 0/306 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/77 [00:00<?, ?it/s]

Training... :   0%|          | 0/306 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/77 [00:00<?, ?it/s]

### Prediction

In [ ]:
test_sents = test['tokenized_sents'].to_numpy()
preds, probs = predict(pt_model, test_sents, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/48 [00:00<?, ?it/s]

(1530,) (1530, 10)


In [ ]:
preds

array([0, 2, 2, ..., 2, 2, 2])

In [ ]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['heg', 'nei', 'nei', 'nei', 'heg']

###F1 Score and report

In [ ]:
y_test, y_pred = test['labels'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.8908405394245961
--------------classification_report---------------
              precision    recall  f1-score   support

         heg       0.90      0.85      0.88       510
         kan       0.86      0.91      0.89       510
         nei       0.92      0.90      0.91       510

    accuracy                           0.89      1530
   macro avg       0.89      0.89      0.89      1530
weighted avg       0.89      0.89      0.89      1530



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[436  48  26]
 [ 28 466  16]
 [ 20  29 461]]


Slightly better results can be seen when using the preprocessed data. Although the size of the train data differs in two cases, in my opinion the preprocessing step has done it's job.<br/>Just to mention, I didn't think philosophical text's would be so much distinguishable from each other. Signs of this fact became somehow obvious when we checked word frequency in each text.